In [2]:
import numpy as np
import pandas as pd

In [16]:
#  let say we have 10 points 
import pandas as pd

# Define data (without header inside)
points = [
    [1, 1.0, 1.2],
    [2, 1.3, 1.0],
    [3, 0.8, 1.5],
    [4, 5.0, 4.8],
    [5, 5.2, 5.1],
]

# Create DataFrame with proper column names
data = pd.DataFrame(points, columns=['ID', 'X', 'Y'])

print(data.iloc[:,1:])


     X    Y
0  1.0  1.2
1  1.3  1.0
2  0.8  1.5
3  5.0  4.8
4  5.2  5.1


In [30]:
def Single_linked():
    pass

def DistanceMatrix(points):
    n=points.shape[0]
    mat=np.zeros((n,n))
    for i in range(n):
        for j in range(i+1):
            mat[i,j]=np.linalg.norm( points.iloc[i,:] - points.iloc[j,:] )
    return mat

print(DistanceMatrix(data.iloc[:,1:]))

[[0.         0.         0.         0.         0.        ]
 [0.36055513 0.         0.         0.         0.        ]
 [0.36055513 0.70710678 0.         0.         0.        ]
 [5.38144962 5.30377224 5.34134814 0.         0.        ]
 [5.73149195 5.65862174 5.68506816 0.36055513 0.        ]]


In [32]:


def Agglomerative_Clustering(data, method='single'):
    """
    Perform agglomerative hierarchical clustering dynamically for different linkage methods.
    
    Parameters:
        data (DataFrame): DataFrame with features in columns (exclude ID column).
        method (str): Linkage method: 'single', 'complete', 'average', 'ward'.
    
    Returns:
        merge_sequence (list): List of merges as tuples of cluster indices.
    """
    points = data.iloc[:, 1:].values
    n = points.shape[0]
    
    # Initialize clusters: each point is its own cluster
    clusters = [[i] for i in range(n)]
    
    # Precompute full distance matrix
    dist_mat = np.linalg.norm(points[:, None, :] - points[None, :, :], axis=2)
    np.fill_diagonal(dist_mat, np.inf)
    
    merge_sequence = []
    
    while len(clusters) > 1:
        min_dist = np.inf
        min_i, min_j = -1, -1
        
        # Find pair of clusters to merge
        for i in range(len(clusters)):
            for j in range(i):
                # Extract all pairwise distances between clusters[i] and clusters[j]
                pair_dists = dist_mat[np.ix_(clusters[i], clusters[j])]
                
                if method == 'single':
                    d = np.min(pair_dists)
                elif method == 'complete':
                    d = np.max(pair_dists)
                elif method == 'average':
                    d = np.mean(pair_dists)
                elif method == 'ward':
                    # Compute increase in within-cluster variance if merged
                    combined = clusters[i] + clusters[j]
                    points_i_j = points[combined, :]
                    d = np.sum((points_i_j - points_i_j.mean(axis=0))**2)
                else:
                    raise ValueError("Unknown method. Choose: 'single', 'complete', 'average', 'ward'.")
                
                if d < min_dist:
                    min_dist = d
                    min_i, min_j = i, j
        
        # Merge the selected clusters
        new_cluster = clusters[min_i] + clusters[min_j]
        merge_sequence.append((clusters[min_i], clusters[min_j]))
        
        # Update cluster list
        clusters = [clusters[k] for k in range(len(clusters)) if k not in (min_i, min_j)]
        clusters.append(new_cluster)
    
    return merge_sequence

# Example usage:
merge_seq_single = Agglomerative_Clustering(data, method='single')
merge_seq_complete = Agglomerative_Clustering(data, method='complete')
merge_seq_avg = Agglomerative_Clustering(data, method='average')
merge_seq_ward = Agglomerative_Clustering(data, method='ward')

print("Single-link merges:", merge_seq_single)
print("Complete-link merges:", merge_seq_complete)


Single-link merges: [([4], [3]), ([1], [0]), ([1, 0], [2]), ([1, 0, 2], [4, 3])]
Complete-link merges: [([4], [3]), ([1], [0]), ([1, 0], [2]), ([1, 0, 2], [4, 3])]
